In [1]:
import time
import random
import smtplib
import socket
import os
from datetime import datetime, timedelta
from collections import defaultdict
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import chromedriver_autoinstaller
import os

import os

# Load SMTP Password from Environment Variables
SMTP_PASSWORD = os.getenv("SMTP_PASSWORD")

if SMTP_PASSWORD is None:
    raise ValueError("SMTP_PASSWORD environment variable is not set.")

print("Password loaded successfully (hidden)")  # Password won't be printed

# SMTP Email Configuration
SMTP_SERVER = "smtp.gmail.com"  # Use your email provider's SMTP server
SMTP_PORT = 587
SMTP_USERNAME = "shivamjoshi70@gmail.com"  # Replace with your email
RECIPIENT_EMAIL = ["shivamjoshi70@gmail.com", "sameerm683@gmail.com"]

 
# Search settings
SEARCH_URL = "https://www.find-tender.service.gov.uk/Search/Results"
KEYWORDS = ['"THAMES WATER UTILITIES LIMITED"', '"SOUTHERN WATER UTILITIES LIMITED"',
                '"SOUTH WEST WATER LIMITED"', '"South West Water Ltd"', '"PENNON GROUP PLC"',
                '"SSE Plc"', '"SSE SERVICES PLC"', '"SEVERN TRENT WATER LIMITED"',
                '"NORTHUMBRIAN WATER GROUP LIMITED"', '"Portsmouth Water Limited"',
                '"AFFINITY WATER LIMITED"', '"Dwr Cymru Welsh Water"',
                '"UK POWER NETWORKS (OPERATIONS) LIMITED"', '"CENTRICA PLC"',
                '"GNI (UK) Limited"', '"UNITED UTILITIES WATER LIMITED"', '"ANGLIAN WATER SERVICES LIMITED"']
OUTPUT_FILENAME = "tenders.txt"
 
# Setup WebDriver (Installs ChromeDriver automatically)
def setup_driver():
    chromedriver_autoinstaller.install()
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in headless mode
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)
    return driver
 
# Open date range filter
def open_date_range_widget(driver):
    try:
        date_range_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Date range')]"))
        )
        date_range_btn.click()
        time.sleep(1.5)
        print("✅ Date range widget opened.")
    except Exception as e:
        print(f"❌ Error opening date range widget: {e}")
 
# Open publication date section
def open_publication_date_section(driver):
    try:
        for _ in range(2):
            pub_date_span = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//span[@aria-label='Publication date']"))
            )
            pub_date_span.click()
            time.sleep(1)
        print("✅ Publication date section opened.")
    except Exception as e:
        print(f"❌ Error opening publication date section: {e}")
 
# Set date filters
def set_date_inputs(driver, prefix, target_date):
    try:
        day_input = driver.find_element(By.NAME, f"{prefix}[day]")
        month_input = driver.find_element(By.NAME, f"{prefix}[month]")
        year_input = driver.find_element(By.NAME, f"{prefix}[year]")
 
        driver.execute_script("arguments[0].value = arguments[1];", day_input, target_date.strftime("%d"))
        driver.execute_script("arguments[0].value = arguments[1];", month_input, target_date.strftime("%m"))
        driver.execute_script("arguments[0].value = arguments[1];", year_input, target_date.strftime("%Y"))
 
        time.sleep(1)
        print(f"✅ {prefix} date set to: {target_date.strftime('%d/%m/%Y')}")
    except Exception as e:
        print(f"❌ Error setting {prefix} date: {e}")
 
# Click update results
def click_update_results(driver):
    try:
        update_btn = driver.find_element(By.ID, "adv_search_button")
        update_btn.click()
        time.sleep(2)
        print("✅ Results updated.")
    except Exception as e:
        print(f"❌ Error updating results: {e}")
 
# Refine search using keyword
def refine_search(driver, keyword):
    try:
        search_box = driver.find_element(By.ID, "keywords")
        search_box.clear()
        search_box.send_keys(keyword)
        time.sleep(1)
        search_box.send_keys(Keys.RETURN)
        time.sleep(2)
        print(f"✅ Search refined: \"{keyword}\"")
    except Exception as e:
        print(f"❌ Search box error: {e}")
 
# Extract tenders
def extract_tenders(driver, max_pages=3):
    tender_links = set()
    for page in range(1, max_pages + 1):
        print(f"\n📄 Fetching page {page}...")
        time.sleep(random.uniform(2, 3))
        try:
            tenders = driver.find_elements(By.XPATH, "//a[contains(@href, '/Notice/')]")
            page_links = {tender.get_attribute("href") for tender in tenders if tender.get_attribute("href")}
            tender_links.update(page_links)
            if not page_links:
                print(f"⏳ No tenders found on page {page}.")
                break
        except Exception as e:
            print(f"❌ Error extracting tenders: {e}")
            break
    return list(tender_links)
 
# Save results
def save_results_to_file(results):
    try:
        with open(OUTPUT_FILENAME, "w", encoding="utf-8") as file:
            for keyword, links in results.items():
                file.write(f"\n🔍 {keyword}:\n" + "\n".join(links) + "\n")
        print(f"📂 Results saved to {OUTPUT_FILENAME}")
    except Exception as e:
        print(f"❌ Error saving results: {e}")
 
# Send email with results
def send_email(subject, body, attachment):
    try:
        msg = MIMEMultipart()
        msg["From"] = SMTP_USERNAME
        msg["To"] = ", ".join(RECIPIENT_EMAIL)
        msg["Subject"] = subject
        msg.attach(MIMEText(body, "plain"))
 
        with open(attachment, "r", encoding="utf-8") as file:
            attachment_text = file.read()
            msg.attach(MIMEText(attachment_text, "plain"))
 
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(SMTP_USERNAME, SMTP_PASSWORD)
        server.sendmail(SMTP_USERNAME, RECIPIENT_EMAIL, msg.as_string())
        server.quit()
 
        print("📧 Email sent successfully!")
    except socket.timeout:
        print("❌ Email error: Connection timeout.")
    except Exception as e:
        print(f"❌ Email error: {e}")
 
# Main function
def main():
    driver = setup_driver()
    results = defaultdict(list)
 
    from_date = datetime.today() - timedelta(days=2)
    to_date = datetime.today()
 
    for keyword in KEYWORDS:
        print(f"\n🔍 Searching for: \"{keyword}\"")
        driver.get(SEARCH_URL)
        time.sleep(2)
 
        open_date_range_widget(driver)
        open_publication_date_section(driver)
        set_date_inputs(driver, "published_from", from_date)
        set_date_inputs(driver, "published_to", to_date)
        click_update_results(driver)
        refine_search(driver, keyword)
 
        results[keyword] = extract_tenders(driver)
 
    driver.quit()
    save_results_to_file(results)
    send_email("Tender Alert", "Attached are the latest tenders.", OUTPUT_FILENAME)
 
if __name__ == "__main__":
    main()

Password loaded successfully (hidden)



🔍 Searching for: ""THAMES WATER UTILITIES LIMITED""


✅ Date range widget opened.


✅ Publication date section opened.


✅ published_from date set to: 08/11/2025


...

✅ published_to date set to: 10/11/2025


✅ Results updated.


✅ Search refined: ""ANGLIAN WATER SERVICES LIMITED""

📄 Fetching page 1...


⏳ No tenders found on page 1.
📂 Results saved to tenders.txt


📧 Email sent successfully!
